In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
%cd H:\Python\BDSL_THESIS\TRAIN
%ls

H:\Python\BDSL_THESIS\TRAIN
 Volume in drive H is SSD
 Volume Serial Number is 1E82-13E0

 Directory of H:\Python\BDSL_THESIS\TRAIN

05/29/2024  08:46 PM    <DIR>          .
05/29/2024  08:45 PM    <DIR>          ..
05/29/2024  08:46 PM    <DIR>          0
05/29/2024  08:46 PM    <DIR>          1
05/29/2024  08:46 PM    <DIR>          2
05/29/2024  08:46 PM    <DIR>          3
05/29/2024  08:46 PM    <DIR>          4
05/29/2024  08:46 PM    <DIR>          5
05/29/2024  08:46 PM    <DIR>          6
05/29/2024  08:46 PM    <DIR>          7
               0 File(s)              0 bytes
              10 Dir(s)  54,725,414,912 bytes free


In [3]:
class_list=['0', '1', '2', '3', '4', '5', '6', '7']
dataset_path = '/Python/BDSL_THESIS/TRAIN/'
image_paths = []
labels = []
for i in class_list:
    for folder_name in os.listdir(dataset_path):
        if folder_name == i:
            folder_path = os.path.join(dataset_path, folder_name)
            image_paths.append(folder_path)

for i in image_paths:
    print(i.rsplit('/', 1)[-1])

0
1
2
3
4
5
6
7


In [4]:
dataset_path = '/Python/BDSL_THESIS/TRAIN/'

In [5]:
train_images = []
train_labels = []
for class_name in os.listdir(dataset_path):
    if class_name not in class_list:
        continue
    class_path = os.path.join(dataset_path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (32,32), interpolation = cv2.INTER_AREA)
        image = image.astype('float32')
        min_val = np.min(image)
        max_val = np.max(image)
        iamge = image - min_val
        image = image / (max_val - min_val)
        train_images.append(image)
        label = int(class_name)
        train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

print(train_images.shape)
print(train_labels.shape)

(2292, 32, 32, 3)
(2292,)


In [6]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_list,
    shuffle=True)

num_classes = train_generator.num_classes
class_indices = train_generator.class_indices
print('Number of classes:', num_classes)
print('Class indices:', class_indices)

Found 2292 images belonging to 8 classes.
Number of classes: 8
Class indices: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7}


In [15]:
cv2.imshow('Image', train_images[299]*1)

print(train_images.shape)
print(train_labels.shape)

(2292, 32, 32, 3)
(2292,)


In [16]:
num_new_images = 2292*2

datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.1,
                             zoom_range=0.1, horizontal_flip=True,
                             fill_mode='nearest')

# Generate new images and labels
new_images = []
new_labels = []
for i in range(num_new_images):
    batch = datagen.flow(train_images, train_labels, batch_size=1)
    new_image, new_label = batch.next()
    new_images.append(new_image[0])
    new_labels.append(new_label[0])

# Convert the images and labels to NumPy arrays
new_images = np.array(new_images)
new_labels = np.array(new_labels)

# Concatenate the new images and labels with the original ones
train_images = np.concatenate([train_images, new_images])
train_labels = np.concatenate([train_labels, new_labels])

In [17]:
np.save('/Python/BDSL_THESIS/npy/train_img.npy', train_images)
np.save('/Python/BDSL_THESIS/npy/train_label.npy', train_labels)

In [19]:
dataset_path = '/Python/BDSL_THESIS/TEST/'
train_images = []
train_labels = []
for class_name in os.listdir(dataset_path):
    if class_name not in class_list:
        continue
    class_path = os.path.join(dataset_path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (32,32), interpolation = cv2.INTER_AREA)
        image = image.astype('float32')
        min_val = np.min(image)
        max_val = np.max(image)
        iamge = image - min_val
        image = image / (max_val - min_val)
        train_images.append(image)
        label = int(class_name)
        train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

print(train_images.shape)
print(train_labels.shape)

(320, 32, 32, 3)
(320,)


In [20]:
np.save('/Python/BDSL_THESIS/npy/test_img.npy', train_images)
np.save('/Python/BDSL_THESIS/npy/test_label.npy', train_labels)